# LIGO/Virgo O3a Pipeline Sensitivity Data Product Tutorial

This document describes the LIGO/Virgo O3a pipeline sensitivity data product that can be used to calculate the selection function describing the LIGO/Virgo searches for binary black holes.  The selection function is a crucial component for any "population analysis" of the GWTC-2 binary black hole population.

To produce this data product, the collaborations have

1. Generated a large number of synthetic binary black hole signals with parameters drawn randomly from a distribution (specified below) over black hole masses, black hole spins, redshift, orientation, etc, etc.  The total number of signals generated is recorded in the data product (see below).
1. Removed any generated signals whose estimated S/N in the detectors fell below a very low threshold (these are "hopeless" for detection in any case).
1. Injected the remaining signals into the O3a data stream (in software!) at uniform intervals in time.
1. Re-searched the modified O3a data stream with the same pipelines used to produce the GWTC-2 catalog and recorded for each injected signal the inverse false alarm rate (a measure of detection significance) calculated in each pipeline.
1. Produced a data file containing the parameters, detection statistics, and the properly-normalized probability density of the draw distribution for every synthetic BBH signal not removed in step 2.

Given a detection statistic threshold, counting the number of injections in the file above the threshold and dividing by the total number of signals generated gives a Monte-Carlo estimate of the *fraction* of the generated population that is detectable by our searches above that threshold.  The fraction of detectable signals from a different population can be estimated by applying weights, given by the ratio between the different population's probability density and the injected population's density, before counting detected injections.  

The remainder of this document describes the sensitivity data product in detail and gives an example of such a re-weighting procedure to estimate the sensitivity of the O3a searches to populations such as those described in the LIGO/Virgo O1/O2 population analysis.

In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'

Populating the interactive namespace from numpy and matplotlib


In [2]:
import astropy.cosmology as cosmo
from astropy.cosmology import Planck15
import astropy.units as u
import h5py
import seaborn as sns

sns.set_context('notebook')
sns.set_palette('colorblind')
sns.set_style('ticks')

## Preliminaries / Definitions

Some useful references are [Loredo (2004)](https://ui.adsabs.harvard.edu/abs/2004AIPC..735..195L/abstract); [Mandel, Farr, & Gair (2018)](https://ui.adsabs.harvard.edu/abs/2019MNRAS.486.1086M/abstract); [Tiwari (2018)](https://ui.adsabs.harvard.edu/abs/2018CQGra..35n5009T/abstract); and [Farr (2019)](https://ui.adsabs.harvard.edu/abs/2019RNAAS...3...66F/abstract).  The development here mostly follows Farr (2019).

We assume that the events whose population we are trying to model occur as a Poisson point process.  Each event has some (latent---that is not perfectly known / observed) parameters $\theta$, and events occur with an intensity (or rate) in parameter space given by
$$
\frac{\mathrm{d} N}{\mathrm{d} \theta}(\lambda),
$$
where $\lambda$ are parameters that determine the intensity.  It is these "population-level" parameters that we are trying to determine by analyzing our catalog.

We do not have access to the parameters of an event directly, but only through some observational data, $d$, which include some stochastic noise such that we can only recover the parameters probabilistically through a likelihood function that describes the distribution of data at fixed parameters:
$$
p\left( d \mid \theta \right).
$$

Further, we assume that a catalog of events is produced by a process that *censors* some of the data---that is, there is some "pipeline" that examines the data and decides whether it should be included in the catalog.  We assume this process is a *deterministic* function of the data (not the parameters $\theta$---which are unknown), so there is some function $f$ and a threshold value $f_\mathrm{th}$ such that the catalog is composed of exactly those data where $f(d) > f_\mathrm{th}$.  ([Mandel, Farr, & Gair (2018)](https://ui.adsabs.harvard.edu/abs/2019MNRAS.486.1086M/abstract) discuss modeling in the case where some of these assumptions are relaxed, particularly when the censoring process is probabilistic rather than deterministic, or when the censoring depends on both data and parameters; an example of this latter situation was discussed in [Farr & Mandel (2018)](https://ui.adsabs.harvard.edu/abs/2018Sci...361.6506F/abstract) where physical processes enforced that some combinations of parameter values will not enter the catalog.)

Under these assumptions, the appropriate likelihood function for a catalog of events with index $i = 1, \ldots, N$ is ([Loredo (2004)](https://ui.adsabs.harvard.edu/abs/2004AIPC..735..195L/abstract); [Mandel, Farr, & Gair (2018)](https://ui.adsabs.harvard.edu/abs/2019MNRAS.486.1086M/abstract))
$$
p\left( \left\{ d_i, \theta_i \mid i = 1, \ldots, N \right\} \mid \lambda \right) = \exp\left( -\Lambda(\lambda) \right) \prod_{i=1}^N p\left( d_i \mid \theta_i \right) \frac{\mathrm{d} N}{\mathrm{d} \theta}(\lambda),
$$
where
$$
\Lambda(\lambda) = \int_{\left\{ d \mid f(d) > f_\mathrm{th} \right\}} \mathrm{d} d \, \mathrm{d} \theta \, p\left( d \mid \theta \right) \frac{\mathrm{d} N}{\mathrm{d} \theta}(\lambda),
$$
is the expected number of objects in the catalog when the population is described by the population parameters $\lambda$ (the actual number of objects in the catalog when the population parameters are $\lambda$ will be Poisson distributed with mean $\Lambda$).  Note that $\Lambda$ is a double integral over both *data* and *parameters* drawn respectively from the likelihood function and the population *subject to the constrant that the data would have passed the censoring threshold for the catalog*.  The selection function enters a population analysis entirely through $\Lambda$.

LIGO/Virgo release a data product that permits importance-weighted Monte-Carlo estimation of $\Lambda$.  The data product is a suite of random binary black hole waveforms that have been injected into the LIGO/Virgo data stream and detected by the same LIGO/Virgo pipelines that have been used to produce the catalog.  LVC have drawn BBH merger params from a fiducial distribution, $p_\mathrm{draw}\left( \theta \right)$ (more details below), used these to generate data from the *empirical* likelihood function, $p\left( d \mid \theta \right)$, filtered through the *actual* pipeline $f(d)$, and recorded the pipeline output statistic for each injection (labelled "IFAR" for Inverse False Alarm Rate; you can set your own "effective" threshold on this number, or use the LVC threshold of $\mathrm{IFAR} > 1 \, \mathrm{yr}$ from its own analysis, [Abbott, et al. (2020)](https://dcc.ligo.org/LIGO-P2000077/public)) which is directly comparable to the same statistic reported for the events in the O3a catalog [(Abbott, et al. 2020)](https://dcc.ligo.org/LIGO-P2000061/public).  LVC also report the *properly normalized* value of $p_\mathrm{draw}\left( \theta \right)$ for each of these injections (again, see below).

Integrals of the form
$$
I[g] = \int_{f(d) > f_\mathrm{th}} \mathrm{d} d \, \mathrm{d} \theta \, p\left( d \mid \theta \right) p_\mathrm{draw} \left( \theta \right) g\left( \theta \right)
$$
can be estimated via the Monte-Carlo method using these LVC data products because
$$
I[g] \simeq \frac{1}{N_\mathrm{draw}} \sum_{f_i > f_\mathrm{th}} g\left( \theta_i \right),
$$
where $N_\mathrm{draw}$ is the total number of draws LVC have made from their fiducial distribution $p_\mathrm{draw}\left( \theta \right)$, and $f_i$ and $\theta_i$ are the IFAR and parameters for the $i$th draw.  The sum is taken over all synthetic injections that are above the threshold $f_\mathrm{th}$.  

$\Lambda(\lambda)$ is just such an integral, with 
$$
g_\Lambda\left(\theta\right) \equiv \frac{1}{p_\mathrm{draw}\left( \theta \right)} \frac{\mathrm{d} N}{\mathrm{d} \theta},
$$
so that 
$$
\Lambda\left( \lambda \right) \simeq \frac{1}{N_\mathrm{draw}} \sum_{f_i > f_\mathrm{th}} \frac{1}{p_\mathrm{draw}\left( \theta_i \right)} \frac{\mathrm{d} N}{\mathrm{d} \theta_i}\left(\lambda \right).
$$
The estimate is effectively an importance-weighted Monte-Carlo estimate of $\Lambda$.

The LVC $p_\mathrm{draw}$ has been chosen to be approximately representative of the observed BBH population observed in O3a.  [Farr (2019)](https://ui.adsabs.harvard.edu/abs/2019RNAAS...3...66F/abstract) gives formulas for estimating the uncertainty in this estimate of $\Lambda$, and gives a threshold below which the computation is sufficiently accurate to avoid bias in population inference.  We reproduce them here (Eqs. (8) and (9)):
$$
\mu \equiv \frac{1}{N_\mathrm{draw}} \sum_{f_i > f_\mathrm{th}} \frac{1}{p_\mathrm{draw}\left( \theta_i \right)} \frac{\mathrm{d} N}{\mathrm{d} \theta_i}\left(\lambda \right)
$$
and
$$
\sigma^2 \equiv \frac{\mu^2}{N_\mathrm{eff}} \equiv \frac{1}{N_\mathrm{draw}^2} \sum_{f_i > f_\mathrm{th}} \left[\frac{1}{p_\mathrm{draw}\left( \theta_i \right)} \frac{\mathrm{d} N}{\mathrm{d} \theta_i}\left(\lambda \right)\right]^2 - \frac{\mu^2}{N_\mathrm{draw}};
$$
under repeated samplings, and for sufficiently large draws in each sampling, estimates of $\Lambda$ will be approximately Normally distributed with mean $\mu$ and variance $\sigma^2$.  The $N_\mathrm{eff}$ parameter measures the "effective sample size" of the injection suite for each particular choice of population parameters $\lambda$ (this will generally be larger for populations similar to the distribution from which the injections have been drawn because the variance of the terms in the sum above will be smaller).

The particular draw distribution chosen for O3a is 
$$
p\left( m_1 \right) \propto m_1^{-2.35},
$$
$$
p\left( m_2 \mid m_1 \right) \propto m_2^2,
$$
$$
p\left( z \right) \propto \left(1 + z \right)^2 \times \left[ \frac{\mathrm{d} V_C}{\mathrm{d} z} \frac{1}{1+z} \right],
$$
and
$$
p\left( s_{1,z} \right) = p\left( s_{2,z} \right) \propto \mathrm{const},
$$
for masses $2 \, M_\odot \leq m_2 \leq m_1 \leq 100 \, M_\odot$, redshift $0 \leq z \leq 2.3$, and orbit-aligned dimensionless spin parameters $-1 \leq s_{1,2,z} \leq 1$ (the injections have zero spin components perpindicular to the orbital plane; they are "aligned spin").  *Once again: the properly-normalized density, 
$$
p_\mathrm{draw}\left( m_1, m_2, z, s_{1,z}, s_{2,z} \right) = p\left( m_1 \right) p\left( m_2 \mid m_1 \right) p\left( z \right) p\left( s_{1,z} \right) p\left( s_{2,z} \right)
$$ 
is reported along with the injection parameters and pipeline IFAR values, so the necessary values to compute the importance-weighted estimate of $\Lambda$---except for the intensity $\mathrm{d}N/\mathrm{d}\theta$, which must be supplied by the consumer in any case---are all present in the released LVC data product.*  

The LVC data product also gives the total amount of "detector time" during which injections were produced; injections are distributed uniformly in time in the detector frame, so an additional multiplicative factor of $T_\mathrm{inj}$ should appear in the estimate of $\Lambda$.  The Poisson intensity carries an additional---often implicit---factor of "per observer time;" so the draw density should also have a factor of "per observer time," which is $1/T$ if the injections are uniform in observer time.  The result is to multiply the importance-weighted estimate by $T$.  This factor is not included in the LVC $p_\mathrm{draw}$ values, but is released in the data product; see below.

## The Sensitivity Data Product

Here we load the samples from the sensitivity data product and make some basic plots with them.  First, the structure of the HDF5 file:

In [3]:
with h5py.File('o3a_bbhpop_inj_info.hdf', 'r') as f:
    print(list(f.keys()))
    print(list(f['injections'].keys()))
    print(list(f.attrs.items()))

['injections']
['distance', 'gps_time_int', 'ifar_gstlal', 'ifar_pycbc_bbh', 'ifar_pycbc_full', 'mass1_source', 'mass2_source', 'optimal_snr_h', 'optimal_snr_l', 'redshift', 'sampling_pdf', 'spin1z', 'spin2z']
[('N_exp/R(z=0)', 979.0426119706958), ('analysis_time_s', 15843600), ('max_redshift', 2.3), ('n_accepted', 156878), ('n_rejected', 76825621), ('surveyed_VT_Gpc_yr', 167.60465680916), ('total_generated', 76982499)]


The defined attributes are:

* `N_exp/R(z=0)` gives the normalizing constant for the redshift part of the distribution, equivalent to the number of mergers expected for the merger intensity (all quantities in the source frame)
$$
\frac{\mathrm{d} N}{\mathrm{d} m_1 \mathrm{d} m_2 \mathrm{d} V_c \mathrm{d} t} = 1 \, \mathrm{Gpc}^{-3} \, \mathrm{yr}^{-1} p\left( m_2 \mid m_1 \right) p\left( m_1 \right) \left( 1 + z \right)^2,
$$
  where the mass distribution integrates to 1 over the allowable range of masses.  In other words, the number of mergers in the injected volume during the observing run if the local merger rate is $1 \, \mathrm{Gpc}^{-3} \, \mathrm{yr}^{-1}$.
* `analysis_time_s` gives the total (detector frame) searched time.
* `max_redshift` gives the maximum redshift over which injections have been performed.
* `n_accepted` gives the number of injections accepted after a first very conservative SNR cut.  Recall that the injection campaign proceeded in two stages for computational efficiency: the first stage cuts "hopeless" injections based on SNR and the second pushed the remaining injections through the detection pipeline.
* `n_rejected` gives the number of injections judged "hopeless" in the first cut and therefore never pushed through the pipeline.
* `surveyed_VT_Gpc_yr` gives the total spacetime volume "surveyed" by the injections (i.e. the comoving time-volume out to the maximum redshift over the observing run).
* `total_generated` is the total number of injections drawn from the reference distribution (= `n_accepted` + `n_rejected`).

The datasets in the `injections` group are 

* `distance` gives the luminosity distance in Mpc for each non-hopeless injection.
* `gps_time_int` gives the integer seconds of the GPS time for the injection.
* `ifar_gstlal` gives the inverse false alarm rate (in years) for that injection's recovery in the `gstlal` pipeline (injections that could not be performed---because, for example, the interferometers were not in observing mode at the time corresponding to the injection---have an IFAR of `-1`, and injections performed but not recovered by the pipeline have an IFAR of `0`).
* `ifar_pycbc_bbh` gives the IFAR (years) in the BBH-specific `PyCBC` pipeline.
* `ifar_pycbc_full` gives the IFAR (years) in the general `PyCBC` pipeline.
* `mass1_source` and `mass2_source` give the source-frame masses for the injection (in solar masses).
* `optimal_snr_h` and `optimal_snr_l` give the optimal (face-on, overhead source) SNR in the H1 and L1 detector using a representative noise spectral density (these were used to define the "hopeless" cut).
* `redshift` gives the injection redshift corresponding to `distance`.
* `sampling_pdf` gives the value of the normalized PDF $p\left( m_1, m_2, z, s_{1,z}, s_{2,z} \right)$ from which the injections are drawn evaluated at each injection's parameters.
* `spin1z` and `spin2z` give the dimensionless orbit-aligned spin components for each injection.

## Example: Estimating Expected Detections in O3a from O1/O2 Population Model

To demonstrate the use of these samples, we will compute the expected number of detections in O3a of a BBH population similar to the ones found to be a good fit to the O1/O2 observations in [Abbott, et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019ApJ...882L..24A/abstract).  We use the power-law models in mass with a cutoff at low and high mass, an evolving redshift distribution, and assume a uniform distribution in the aligned spin components; see particularly Section 4.2 and Figure 6 for the discussion of redshift evolution.

Since it is common in population analysis, we work with the log of $\mathrm{d}N/\mathrm{d}\theta$.

The ultimate population model is (all quantities in the comoving frame)
$$
\frac{\mathrm{d} N}{\mathrm{d} m_1 \mathrm{d} m_2 \mathrm{d} V \mathrm{d} t} = 
\begin{cases}
\frac{R}{C\left( \alpha, \beta \right)} m_1^{-\alpha} m_2^\beta \left( 1 + z \right)^\gamma & M_\mathrm{min} < m_2 < m_1 < M_\mathrm{max} \\
0 & \mathrm{otherwise}
\end{cases}
$$
where $C\left(\alpha, \beta\right)$ is a normalizing constant ensuring that the integral over all masses is unity, so that 
$$
\frac{\mathrm{d} N}{\mathrm{d} V \mathrm{d} t} = \int \mathrm{d} m_1 \, \mathrm{d} m_2 \, \frac{\mathrm{d} N}{\mathrm{d} m_1 \mathrm{d} m_2 \mathrm{d} V \mathrm{d} t} = R \left( 1 + z \right)^\gamma.
$$
Thus we see that $R$ is the merger rate of BBHs at all masses at $z = 0$.

In [4]:
R0 = 20.0 # per Gpc^3 per yr

alpha = 1.8 # primary mass pl slope
beta = 0 # secondary mass pl slope
gamma = 2.7 # redshift slope

mmin = 5 # Solar masses
mmax = 41 # Solar masses

def log_dNdm1dm2dzdsz2(m1, m2, z, s1z, s2z):
    m1_norm = (1-alpha)/(mmax**(1-alpha) - mmin**(1-alpha))
    m2_norm = (1+beta)/(m1**(1+beta) - mmin**(1+beta))
    
    log_pm1 = -alpha*log(m1) + log(m1_norm)
    log_pm2 = beta*log(m2) + log(m2_norm)
    
    # Here we convert from dN/dVdt in the comoving frame to dN/dzdt in the detector frame:
    #
    # dN/dzdt = dN/dVdt * dV/dz * 1/(1+z)
    #
    # The first factor is the Jacobian between comoving volume and redshift 
    # (commonly called the "comoving volume element"), and the second factor accounts for 
    # time dilation in the source frame relative to the detector frame.
    log_dNdV = log(R0) + gamma*log1p(z)
    log_dVdz = log(4*pi) + log(Planck15.differential_comoving_volume(z).to(u.Gpc**3/u.sr).value)
    log_time_dilation = -log1p(z)
    log_dNdz = log_dNdV + log_dVdz + log_time_dilation
    
    log_p_sz = log(0.25) # 1/2 for each spin dimension
    
    return np.where((m2 < m1) & (mmin < m2) & (m1 < mmax), log_pm1 + log_pm2 + log_p_sz + log_dNdz, np.NINF)

Here we implement the above formulas for (the log of) $\mu$ and $N_\mathrm{eff}$, using as our detection threshold that any of the three searches (`gstlal` and the `PyCBC` targeted BBH and un-targeted general) found the event with a false alarm rate that is below (better than) $1 / \mathrm{yr}$---an "IFAR" (inverse FAR) greater than 1.  This is the same threshold used in [Abbott, et al. (2020](https://dcc.ligo.org/LIGO-P2000077/public).

In [5]:
with h5py.File('o3a_bbhpop_inj_info.hdf', 'r') as f:
    Tobs = f.attrs['analysis_time_s']/(365.25*24*3600) # years
    Ndraw = f.attrs['total_generated']
    
    m1 = array(f['injections/mass1_source'])
    m2 = array(f['injections/mass2_source'])
    z = array(f['injections/redshift'])
    s1z = array(f['injections/spin1z'])
    s2z = array(f['injections/spin2z'])
    
    p_draw = array(f['injections/sampling_pdf'])
    
    gstlal_ifar = array(f['injections/ifar_gstlal'])
    pycbc_ifar = array(f['injections/ifar_pycbc_full'])
    pycbc_bbh_ifar = array(f['injections/ifar_pycbc_bbh'])

In [6]:
one_year = 1.0 # IFAR measured in years
detection_selector = (gstlal_ifar > one_year) | (pycbc_ifar > one_year) | (pycbc_bbh_ifar > one_year)

Note that we select only the detected population according to our threshold using `np.where`.

In [7]:
log_dN = np.where(detection_selector, log_dNdm1dm2dzdsz2(m1, m2, z, s1z, s2z), np.NINF)
log_mu = log(Tobs) + logaddexp.reduce(log_dN - log(p_draw)) - log(Ndraw)

<ipython-input-4-413a3df91e2f>:15: RuntimeWarning: invalid value encountered in log
  log_pm2 = beta*log(m2) + log(m2_norm)


In [8]:
print('This population model would predict that the expected number of detections in O3a is {:.2f}'.format(exp(log_mu)))

This population model would predict that the expected number of detections in O3a is 28.59


Given that we saw ~40 detections in O3a, this population model is reasonable enough.

What is the uncertainty on the above estimate due to the importance weighted Monte-Carlo integral?  Below we compute it following the formulas above.

In [9]:
def logdiffexp(x, y):
    return x + np.log1p(exp(y-x))

log_s2 = 2.0*log(Tobs) + logaddexp.reduce(2.0*log_dN - 2.0*log(p_draw)) - 2.0*log(Ndraw)
log_sigma2 = logdiffexp(log_s2, 2.0*log_mu - log(Ndraw))
Neff = exp(2.0*log_mu - log_sigma2)

In [10]:
print('Uncertainty on sensitivity estimate equivalent to {:.1f} Monte-Carlo samples'.format(Neff))
print('Or, equivalently, a fractional uncertainty on mean detections in O3a of {:.4f}'.format(1/sqrt(Neff)))
print('Or, equivalently, that the expected number of detections in O3a assuming this population is {:.2f} +/- {:.2f}'.format(exp(log_mu), exp(log_mu)/sqrt(Neff)))

Uncertainty on sensitivity estimate equivalent to 13270.0 Monte-Carlo samples
Or, equivalently, a fractional uncertainty on mean detections in O3a of 0.0087
Or, equivalently, that the expected number of detections in O3a assuming this population is 28.59 +/- 0.25


The uncertainty on this estimate is equivalent to a Monte-Carlo integration with $\sim 13$k independent samples (note helpful function `logdiffexp` that computes accurately $\log\left( \exp(x) - \exp(y) \right)$).